In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,GlobalAveragePooling2D, Dropout
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical

C:\Users\Arpit Shukla\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Arpit Shukla\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Arpit Shukla\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)
Using TensorFlow backend.
C:\Users\Arpit Shukla\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Arpit Shukla\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint

In [2]:
#Convolutional neural network
def pure_cnn():
    
    model = Sequential()
    
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same', input_shape=(64,64,3)))    
    model.add(Dropout(0.2))
    
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same'))  
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same', strides = 2))    
    model.add(Dropout(0.5))
    
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))    
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same', strides = 2))    
    model.add(Dropout(0.5))    
    
    model.add(Conv2D(192, (3, 3), padding = 'same'))
    model.add(Activation('relu'))
    model.add(Conv2D(192, (1, 1),padding='valid'))
    model.add(Activation('relu'))
    model.add(Conv2D(4, (1, 1), padding='valid'))

    model.add(GlobalAveragePooling2D())
    
    model.add(Activation('softmax'))

    model.summary()
    
    return model

In [6]:
if __name__ == '__main__':

  target = pd.read_csv('./train.truth.csv')
  names = target['fn']

  #get all images from the dataset(same indexes as target['fn'])
  images = []
  for name in names:
      aux = './train/' + name
      images.append(cv2.imread(aux,1))

  #encode directions 
  LE = LabelEncoder()
  target = LE.fit_transform(target['label'])
  target = np.array(target)


In [9]:
  #split data intro train and test
train_x,test_x,train_y,test_y = train_test_split(images,target,test_size = 0.33)
  train_x = np.array(train_x)
train_y = np.array(train_y)
  test_x = np.array(test_x)
test_y = np.array(test_y)

  #one hot encoding
train_y = to_categorical(train_y)
  test_y = to_categorical(test_y)

train_x = train_x.astype('float32')
  train_y = train_y.astype('float32')
test_x = test_x.astype('float32')
  test_y = test_y.astype('float32')


In [11]:
#normalization of input
train_x /= 255
test_x /= 255


In [14]:
#create model
model = pure_cnn()
  
model.compile(loss='categorical_crossentropy',
                optimizer=Adam(lr=0.0001), # LR = learning rate
                metrics = ['accuracy']) # Metrics to be evaluated by the model

model_details = model.fit(train_x, train_y,
                      batch_size = 32,
                      epochs = 10, # number of iterations
                      validation_data= (test_x,test_y),
                      verbose=1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 64, 64, 96)        2688      
_________________________________________________________________
dropout_4 (Dropout)          (None, 64, 64, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 64, 64, 96)        83040     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
dropout_5 (Dropout)          (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 192)       166080    
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 192)      

In [15]:
 model.save('model.h5')
